__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
avito_path = 'sevastopol/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-12 13:39:36 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-12 13:39:36 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-12 13:39:39 | AdsListParser: downloader init
[INFO    ] 2022-09-12 13:39:39 | AvitoParser: init
[INFO    ] 2022-09-12 13:39:45 | AvitoParser: 74 pages for read
[INFO    ] 2022-09-12 13:39:45 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-12 13:39:48 | AdsListParser: read page 1
[INFO    ] 2022-09-12 13:39:52 | AdsListParser: read page 2
[INFO    ] 2022-09-12 13:39:56 | AdsListParser: read page 3
[INFO    ] 2022-09-12 13:39:59 | AdsListParser: read page 4
[INFO    ] 2022-09-12 13:40:03 | AdsListParser: read page 5
[INFO    ] 2022-09-12 13:40:07 | AdsListParser: read page 6
[INFO    ] 2022-09-12 13:40:11 | AdsListParser: read page 7
[INFO    ] 2022-09-12 13:40:15 | AdsListParser: read page 8
[INFO    ] 2022-09-12 13:40:19 | AdsListParser: read page 9
[INFO    ] 2022-09-12 13:40:23 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3682
3106


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
677,2161799926,"Объявление «1-к. квартира, 29 м², 1/2 эт.» 22 фотографии",4600000,,"ул. Коли Пищенко, 20А","Цена снижена! Нашли встречку. Однокомнатная сталинка с хорошим ремонтом с большой придомовой территорией своей для отдыха. \nВ зеленом районе со всей инфраструктурой. \nПоликлиники, школы, сады, рынок, ТЦ, спортшкола, детский парк «Лукоморье», все в шaгoвой",17,2022-09-12 13:44:49.319330
2849,2352506541,"Объявление «1-к. квартира, 40,1 м², 6/9 эт.» 7 фотографий",9900000,,"Античный пр-т, 5","Арт. 19590900 Продам квартиру в новом доме по адресу: г Севастополь ул. Проспект Античный 5 в ЖК «Легенда».\n\nВ непосредственной близости от дома расположено всё необходимое для комфортной жизни: ТЦ Апельсин, магазины с товарами первой необходимости, 2 новы",69,2022-09-12 13:44:49.319330
